https://en.wikipedia.org/wiki/Tf%E2%80%93idf

In [14]:
import math
import os
import fnmatch # https://docs.python.org/3/library/fnmatch.html

Source https://stevenloria.com/tf-idf/ <BR>

Caveat: this post now uses TextBlob for breaking up the text into words and getting the word counts.

In [15]:
def term_freq(term, list_of_words_in_document):
    """
    computes "term frequency" which is the number of times a word appears in a document, 
    normalized by dividing by the total number of words in document. 
    """
    return list_of_words_in_document.count(term)/(len(list_of_words_in_document)*1.0)

def number_of_documents_containing(term,all_documents):
    """
    Returns the number of documents containing word. 
    """
    countr=0
    for this_doc in all_documents:
        if (term in this_doc):
            countr+=1
    return countr

def inverse_doc_freq(term, all_documents):
    """
    computes "inverse document frequency" which measures 
    how common a word is among all documents in corpus. 
    The more common a word is, the lower its idf. 
    Take the ratio of the total number of documents to the number of documents containing word, 
    then take the log of that. Add 1 to the divisor to prevent division by zero.
    """
    return math.log(len(all_documents) / ( 1.0 + number_of_documents_containing(term, all_documents)))

def tfidf(term, list_of_words_in_document, all_documents):
    """
    computes the TF-IDF score. It's the product of tf and idf.
    """
    return term_freq(term, list_of_words_in_document) * inverse_doc_freq(term, all_documents)

The \*.dat files in the directory have only key words from each file

Convert the .dat contents to lists per document

In [26]:
import math
from textblob import TextBlob as tb

def tf(word, blob):
    return blob.words.count(word) / len(blob.words)

def n_containing(word, bloblist):
    return sum(1 for blob in bloblist if word in blob.words)

def idf(word, bloblist):
    return math.log(len(bloblist) / (1 + n_containing(word, bloblist)))

def tfidf(word, blob, bloblist):
    return tf(word, blob) * idf(word, bloblist)

document1 = tb("""Python is a 2000 made-for-TV horror movie directed by Richard
Clabaugh. The film features several cult favorite actors, including William
Zabka of The Karate Kid fame, Wil Wheaton, Casper Van Dien, Jenny McCarthy,
Keith Coogan, Robert Englund (best known for his role as Freddy Krueger in the
A Nightmare on Elm Street series of films), Dana Barron, David Bowe, and Sean
Whalen. The film concerns a genetically engineered snake, a python, that
escapes and unleashes itself on a small town. It includes the classic final
girl scenario evident in films like Friday the 13th. It was filmed in Los Angeles,
 California and Malibu, California. Python was followed by two sequels: Python
 II (2002) and Boa vs. Python (2004), both also made-for-TV films.""")

document2 = tb("""Python, from the Greek word (πύθων/πύθωνας), is a genus of
nonvenomous pythons[2] found in Africa and Asia. Currently, 7 species are
recognised.[2] A member of this genus, P. reticulatus, is among the longest
snakes known.""")

document3 = tb("""The Colt Python is a .357 Magnum caliber revolver formerly
manufactured by Colt's Manufacturing Company of Hartford, Connecticut.
It is sometimes referred to as a "Combat Magnum".[1] It was first introduced
in 1955, the same year as Smith &amp; Wesson's M29 .44 Magnum. The now discontinued
Colt Python targeted the premium revolver market segment. Some firearm
collectors and writers such as Jeff Cooper, Ian V. Hogg, Chuck Hawks, Leroy
Thompson, Renee Smeets and Martin Dougherty have described the Python as the
finest production revolver ever made.""")

bloblist = [document1, document2, document3]
for i, blob in enumerate(bloblist):
    print("Top words in document {}".format(i + 1))
    scores = {word: tfidf(word, blob, bloblist) for word in blob.words}
    sorted_words = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    for word, score in sorted_words[:3]:
        print("\tWord: {}, TF-IDF: {}".format(word, round(score, 5)))

Top words in document 1
	Word: python, TF-IDF: 0.01662
	Word: films, TF-IDF: 0.00997
	Word: made-for-TV, TF-IDF: 0.00665
Top words in document 2
	Word: genus, TF-IDF: 0.02192
	Word: 2, TF-IDF: 0.02192
	Word: from, TF-IDF: 0.01096
Top words in document 3
	Word: Colt, TF-IDF: 0.01367
	Word: Magnum, TF-IDF: 0.01367
	Word: revolver, TF-IDF: 0.01367


Sample sizes are small, so results are not reliable representations of the document